# Webcrawler practice two
Using this code, I want to get all pictures and their information displayed on-line.<br>
Beacause the code of this web also contains javascript, I use same tool as previous work. <br>
<br>

Selenium<br>
Reference: <br>
&emsp;&emsp; http://selenium-python.readthedocs.io/ <br>
&emsp;&emsp; http://selenium-python-zh.readthedocs.io/en/latest/installation.html (Chinese) <br>
&emsp;&emsp; install: pip install selenium

BeautifulSoup <br>
Reference: <br>
&emsp;&emsp;https://www.crummy.com/software/BeautifulSoup/bs4/doc/ <br>
&emsp;&emsp;install: pip install beautifulsoup4<br>

All I did was to <br>
&emsp;&emsp; 1) create an empty dataframe. <br>
&emsp;&emsp; 2) use selenium to browse the target web. <br>
&emsp;&emsp; 3) select the first category that I want to crawl <br>
&emsp;&emsp; 4) find the max page of this category <br>
&emsp;&emsp; 5) use for loop to get all data and store into the empty dataframe <br>

In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
import pandas as pd #use to form dataframe

In [359]:
df = pd.DataFrame(columns=range(0,5))
URI='http://collections.culture.tw/ntmofa_collectionsweb/tw2/01_search.aspx'
driver=webdriver.Chrome()
driver.get(URI)
# driver.find_element_by_name("cboAATAB1").click()
driver.find_element_by_xpath("//*[@id=\"cboAATAB1\"]/option[2]").click()
driver.find_element_by_xpath("//*[@id=\"btnSearch\"]").click()
    
soup = BeautifulSoup(driver.page_source, "lxml")
jsend ="javascript:__doPostBack('GridView1','Page$Last')"
driver.execute_script(jsend)
soup = BeautifulSoup(driver.page_source, "lxml")
max_page = int(soup.find_all('table')[-1].find_all('td')[-1].get_text())
page = 0
while page <= max_page:
    page += 1
    js ="javascript:__doPostBack('GridView1','Page$" + str(page) +"')"
    driver.execute_script(js)
    soup = BeautifulSoup(driver.page_source, "lxml")
    table = soup.find_all('table')
    table = table[1]
    table = table.find_all('tr')
    total_num = int((len(table)-5)/2)
    new_table = pd.DataFrame(columns=range(0,5), index = [i for i in range(0,total_num+1)])
    row = 0

    for i in range(1,len(table)-2,2):
    #     print(i)
        columns = table[i].find_all('td')
        column_marker = 0
        for column_number in range(4,9):
            new_table.iat[row, column_marker] = columns[column_number].get_text()
            column_marker += 1
    #     print(i,row,column_marker)
        row = row+1

    df = pd.concat([df,new_table],axis=0, ignore_index=True)

In [360]:
df.head(10)

,0,1,2,3,4
0,07200003,\n抽象畫\n,水墨,1979,趙無極
1,07400014,\n梅石圖\n,水墨,1916,吳昌碩
2,07400015,\n竹溪圖\n,水墨,1971,朱念慈
3,07400016,\n西遷圖之二\n,水墨,1947,梁鼎銘
4,07400017,\n螃蟹\n,水墨,1934,齊白石
5,07600001,\n林園之一\n,水墨,1980,黃才松
6,07600002,\n晚翠\n,水墨,1980,戴武光
7,07600003,\n臺中公園所見\n,水墨,1983,蕭進興
8,07600004,\n溪山幽居\n,水墨,1982,程錫牙
9,07600005,\n榕蔭消夏\n,水墨,1983,黃冬富


In [361]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1418 entries, 0 to 1417
Data columns (total 5 columns):
0    1418 non-null object
1    1418 non-null object
2    1418 non-null object
3    1418 non-null object
4    1418 non-null object
dtypes: object(5)
memory usage: 55.5+ KB
